<br><br><br><br>
<span style="color:red;font-size:60px">Charts and graphs</span>
<br><br>
<li><span style="color:green">matplotlib </span>Python's integrated plotting library</li>
<li><span style="color:green">bokeh </span>Interactive plotting library</li>


<br><br><br><br>
<span style="color:blue;font-size:large">matplotlib</span>
<li><a href="https://matplotlib.org">https://matplotlib.org</a></li>
<li>easy to use and tight coupling with pandas</li>


<br><br><br><br>
<span style="color:blue;font-size:large">bokeh</span>
<li><a href="https://docs.bokeh.org/en/latest/">https://docs.bokeh.org/en/latest/</a></li>
<li>interactive plotting, growing mapping functionality, presentation ready graphs</li>
<li>requires a bit more work</li>


<br><br><br><br>
<span style="color:blue;font-size:large">What kinds of visuals are possible</span>
<li>Tons and tons</li>
<li>Best approach: think of how you want to show your data or results and then google till you find the right way to do it</li>
<li><b>Almost anything is possible!</b></li>

<br><br><br><br>
<span style="color:green;font-size:xx-large">setup</span>
<br><br>
<li><b>Important</b>: We will use version 2.4.3 rather than the latest (3.0.3). The latter has bugs working with Jupyter notebooks (set to be fixed in version 3.1)</li>
<li>!pip install bokeh</li>
<li>OR</li>
<li>!conda install --yes bokeh</li>
<li>matplotlib comes pre-installed with anaconda</li>
<li>In the notebook, select "File --> Trust notebook" from the menubar. Your graphs will be available when you reopen the notebook

In [2]:
!pip install bokeh==2.4.3

In [3]:
import bokeh
bokeh.__version__ 

'2.4.3'

<br><br><br><br>
<span style="color:green;font-size:50px">Interactive charts with bokeh</span>
<br><br>

<br><br><br><br>
<span style="color:blue;font-size:large">bokeh</span>
<li>import libraries</li>
<li>run output_notebook() to display charts inline</li>
<li>output_notebook() should say "BokehJS 2.4.3 successfully loaded." (or something went wrong!)</li>

In [4]:
#output_notebook for inline charts in a notebook
#show to show a chart
from bokeh.io import output_notebook, show 
from bokeh.plotting import figure

In [5]:
output_notebook()

Loading BokehJS ...

<br><br><br><br>
<span style="color:blue;font-size:large">Construct a figure</span>
<li>specify labels</li>
<li>specify legend</li>
<li>give a title</li>

<br><br><br><br>
<span style="color:blue;font-size:large">A simple single line plot</span>
<li>first create a figure object</li>
<li>create a line object in the figure object</li>
<li>the line object must contain both axes! Bokeh does not make good use of pandas series</li>
<li>then show the figure


<h3 style="color:blue">The data</h3>
<li>The NYC 311 data subset that we used for maps</li>

In [6]:
import pandas as pd
df = pd.read_csv("../class-datasets/nyc_311_clean.csv",nrows=1000000)
df['Created Date'] = pd.to_datetime(df['Created Date'],format="%Y-%m-%d %H:%M:%S")
df['Closed Date'] = pd.to_datetime(df['Closed Date'],format="%Y-%m-%d %H:%M:%S")
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype         
---  ------           --------------    -----         
 0   Created Date     1000000 non-null  datetime64[ns]
 1   Closed Date      1000000 non-null  datetime64[ns]
 2   Agency           1000000 non-null  object        
 3   Incident Zip     1000000 non-null  int64         
 4   Borough          1000000 non-null  object        
 5   Latitude         1000000 non-null  float64       
 6   Longitude        1000000 non-null  float64       
 7   processing_days  1000000 non-null  float64       
dtypes: datetime64[ns](2), float64(3), int64(1), object(2)
memory usage: 61.0+ MB


<h3>Calculate mean processing time by month and day of week</h3>
<li>Pandas <b>Grouper</b> constructs a groupby instruction for groupby</li>
<li>Useful when dealing with timeseries data</li>
<li>https://pandas.pydata.org/docs/reference/api/pandas.Grouper.html</li>

In [7]:
import pandas as pd

month_groups = df.groupby(pd.Grouper(freq='M',key="Created Date"))
months_mean = month_groups.mean(numeric_only=True)['processing_days']


In [8]:
months_mean

Created Date
2010-04-30    4436.334722
2010-05-31    4351.531250
2010-06-30    4301.313889
2010-07-31    4382.207407
2010-08-31            NaN
                 ...     
2022-02-28       7.989502
2022-03-31       6.005059
2022-04-30       5.280882
2022-05-31       4.910923
2022-06-30       5.406646
Freq: M, Name: processing_days, Length: 147, dtype: float64

In [9]:
day_of_week_groups = df.groupby(df["Created Date"].dt.dayofweek)
day_of_week_mean = day_of_week_groups.mean(numeric_only=True)['processing_days']
day_of_week_mean

Created Date
0    7.272320
1    7.345536
2    8.150530
3    7.595277
4    7.388034
5    5.666127
6    4.096463
Name: processing_days, dtype: float64

<h3>Graph month means and day of week means</h3>

In [10]:
p = figure(title="Monthly Processing Time Averages", x_axis_label='Month', y_axis_label='Processing Time')
p.line(months_mean.index,months_mean,line_width=8,line_color="red")
show(p)

<li>bokeh treats ticks as discrete points and doesn't automatically draw a continuous line</li>
<li>we need three steps for a basic graph rather than one (more complicated than matplotlib)</li>
<li>but we get some nice interactive features along with the graph</li>
<li>we'll need to do some more customization</li>

<li>bokeh can handle time nicely. Convert x-axis into time</li>
<li>tell bokeh that the x-axis is of datetime type. bokeh will assume it is continuous</li>
<li>add gizmos to make the graph look nice</li>

In [11]:
months_mean.index

DatetimeIndex(['2010-04-30', '2010-05-31', '2010-06-30', '2010-07-31',
               '2010-08-31', '2010-09-30', '2010-10-31', '2010-11-30',
               '2010-12-31', '2011-01-31',
               ...
               '2021-09-30', '2021-10-31', '2021-11-30', '2021-12-31',
               '2022-01-31', '2022-02-28', '2022-03-31', '2022-04-30',
               '2022-05-31', '2022-06-30'],
              dtype='datetime64[ns]', name='Created Date', length=147, freq='M')

In [12]:
import pandas as pd
p = figure(title="Mean Processing Time (monthly)",plot_width=600, plot_height=400,
          x_axis_label = "months",y_axis_label = "time",x_axis_type="datetime")

p.line(months_mean.index,#pd.to_datetime(months_mean.index,format="%Y%m"), 
       months_mean,
       legend_label="monthly mean",
      line_color = "red",
      line_width = 8)

show(p)

<br><br><br><br>
<span style="color:blue;font-size:large">tools</span>
<li>bokeh provides a number of interactive tools</li>
<li>in the above two graphs, we let it pick the tools (defaults), but we can control the tools list</li>
<li><a href="https://docs.bokeh.org/en/latest/docs/user_guide/tools.html">tool configuration</a>

In [13]:
import pandas as pd
p = figure(title="Mean Processing Time (monthly)",plot_width=600, plot_height=400,
          x_axis_label = "months",y_axis_label = "time",x_axis_type="datetime",
          tools="pan,box_zoom, crosshair,reset, save")

p.line(pd.to_datetime(months_mean.index,format="%Y%m"), 
       months_mean,
       legend_label="monthly mean",
      line_color = "red",
      line_width = 8)

show(p)



<br><br><br><br>
<span style="color:blue;font-size:large">The bar chart</span>
<li><a href="https://docs.bokeh.org/en/latest/docs/reference/models/glyphs/vbar.html">vbar</a> for vertical bars (hbar for horizontal bars)</li>
<li>x axis labels need to be of type string</li>
<li>We can convert the day numbers to str or just create the label list</li>


In [14]:
day_of_week_mean

Created Date
0    7.272320
1    7.345536
2    8.150530
3    7.595277
4    7.388034
5    5.666127
6    4.096463
Name: processing_days, dtype: float64

In [15]:
number_labels = [str(day) for day in day_of_week_mean.index] #But we'll use text_labels
text_labels = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
value = day_of_week_mean

p = figure(x_range=text_labels, y_range=(0, 25),plot_width=600, plot_height=300)

p.vbar(x=text_labels, top=value , width=0.6, color = "blue",legend_label="means")

show(p)


<br><br><br><br>
<span style="color:blue;font-size:large">Grid plots in bokeh</span>
<li>define each figure separately</li>
<li>use gridplot to create a grid</li>
<li>gridplot takes a matrix of plots as its input</li>

In [16]:
from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
import pandas as pd


p1 = figure(title="Mean Processing Time (monthly)",plot_width=600, plot_height=400,
          x_axis_label = "months",y_axis_label = "time",x_axis_type="datetime",
          tools="pan,box_zoom, crosshair,reset, save")

p1.line(pd.to_datetime(months_mean.index,format="%Y%m"), 
       months_mean,
       legend_label="monthly mean",
      line_color = "red",
      line_width = 8)



text_labels = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
values = day_of_week_mean

p2 = figure(x_range=text_labels, y_range=(0, 25),plot_width=600, plot_height=400)

p2.vbar(x=text_labels, top=values , width=0.5, color = "cyan",legend_label="means")




grid = gridplot([[p1,p2]],sizing_mode="scale_both",merge_tools=True)



show(grid)





<h2>Pie charts</h2>
<span style="color:blue;font-size:large">Breakdown of complaints by Borough</span>
<li>We might be interested in seeing if there are differences in complaint percent across boroughs</li>
<li>Let's make a set of 5 piecharts, one for each borough, that shows the relative number of complaints by agency</li>

<li>roup the data by borough and then, within each borough, by agency, and get the size of each group</li>


In [17]:
df.groupby(['Borough','Agency']).size()

Borough        Agency
BRONX          DCA         387
               DEP        3286
               DHS         290
               DOB        4098
               DOE          83
                         ...  
STATEN ISLAND  DPR        1834
               DSNY      14354
               HPD        2824
               NYPD      13988
               OTI           1
Length: 61, dtype: int64

<li>this gives us a series with a two-layered index</li>
<li>the <a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.unstack.html">unstack</a> function takes a two layered index and converts it into a dataframe</li>
<li>the outer index is the index of the new dataframe</li>
<li>the inner index becomes the columns of the new dataframe</li>
<li>We need to remove the agencies with a NaN in the data (FDNY)</li>

In [18]:
df.groupby(['Borough','Agency']).size().unstack()

Agency,DCA,DEP,DHS,DOB,DOE,DOHMH,DOT,DPR,DSNY,HPD,NYPD,OTI,TLC
Borough,,,,,,,,,,,,,
BRONX,387.0,3286.0,290.0,4098.0,83.0,2884.0,13024.0,2112.0,12694.0,59131.0,142141.0,2.0,NaN
BROOKLYN,522.0,7373.0,1925.0,11737.0,104.0,7077.0,20677.0,5788.0,41712.0,48550.0,153693.0,5.0,NaN
MANHATTAN,470.0,4996.0,4087.0,7225.0,158.0,5578.0,11883.0,1952.0,13871.0,34887.0,97912.0,12.0,1.0
QUEENS,572.0,6148.0,692.0,7214.0,63.0,3933.0,22152.0,4768.0,31814.0,20527.0,135630.0,3.0,NaN
STATEN ISLAND,72.0,1994.0,76.0,1173.0,58.0,784.0,6999.0,1834.0,14354.0,2824.0,13988.0,1.0,NaN


In [19]:
#dropna(axis=1) drops any column that has a nan in it
size_df = df.groupby(['Borough','Agency']).size().unstack().dropna(axis=1)
size_df

Agency,DCA,DEP,DHS,DOB,DOE,DOHMH,DOT,DPR,DSNY,HPD,NYPD,OTI
Borough,,,,,,,,,,,,
BRONX,387.0,3286.0,290.0,4098.0,83.0,2884.0,13024.0,2112.0,12694.0,59131.0,142141.0,2.0
BROOKLYN,522.0,7373.0,1925.0,11737.0,104.0,7077.0,20677.0,5788.0,41712.0,48550.0,153693.0,5.0
MANHATTAN,470.0,4996.0,4087.0,7225.0,158.0,5578.0,11883.0,1952.0,13871.0,34887.0,97912.0,12.0
QUEENS,572.0,6148.0,692.0,7214.0,63.0,3933.0,22152.0,4768.0,31814.0,20527.0,135630.0,3.0
STATEN ISLAND,72.0,1994.0,76.0,1173.0,58.0,784.0,6999.0,1834.0,14354.0,2824.0,13988.0,1.0


<li>each row of the df will correspond to one piechart and we will have 5 piecharts in all</li>

<br><br><br><br>
<span style="color:blue;font-size:large">The pie chart</span>
<li>Drawing a piechart involves specifying each slice of the pie</li>
<li>The <span style="color:blue">color</span> of the slices is defined by importing a <a href="https://docs.bokeh.org/en/latest/docs/reference/palettes.html">color palette</a></li>
<li>We need to use a <a href="https://docs.bokeh.org/en/latest/docs/user_guide/data.html">ColumnDataSource</a> object to store the data:</li>
<ul>
    <li>The data repository for a bokeh chart</li>
    <li>Bokeh can directly reference named columns in this source</li>
    <li>Think of it as a dedicated table that bokeh understands</li>
</ul>
<li><a href="https://docs.bokeh.org/en/latest/docs/user_guide/annotations.html">LabelSet</a>: the set of labels that we want to display. For the piechart, these will be the fraction of cases that are associated with an agency</li>



In [34]:
from math import pi #Need this - a pie chart is a circle
import pandas as pd
import numpy as np
from bokeh.palettes import Category20c #The color palette
from bokeh.models import LabelSet, ColumnDataSource
from bokeh.transform import cumsum #a bokeh function that cumulatively sums the angle column in a column data source

borough = "BRONX"

#Create a dataframe with an Agency column and a value column for the borough
data = size_df.loc[borough].reset_index(name='value')

#We'll restrict the size of the string for Agencies. One Agency is way too long!
data['Agency'] = data['Agency'].str.slice(0,8)

#Calculate a percent column (the fraction of cases for each agency)
data['pct'] = (data['value']/sum(data['value'])*100).round(2)

#Calculate an angle column (the fraction of 360 degrees for each slice)
#Note that data['pct'].sum() should be approx 100.0
#The angle is calculated in radians. 360degrees = 2*pi (approx 6.28)
data['angle'] = data['pct']/(data['pct'].sum()) * 2*pi

#Specify the color palette
#Category20c[len(data)] returns the necessary number of colors from the 20C palette
data['color'] = Category20c[len(data)]

#Define a figure object
#tooltips maps to the ColumnDataSource object
#We haven't created it yet but Agency will map to the Agency column and pct to the pct
#And we'll see NYPD; 50.18 when we hover over the NYPD slice in the piechart
p = figure(plot_height=500, title="Cases by Agency: " + borough, 
        tools="hover", tooltips="@Agency: @pct",x_range=(-0.5, 1.0))

#Add the definition of each wedge
#Each wedge, starting with 0 degrees, will be at the sum of the angles upto that wedge

p.wedge(x=0, y=1, radius=0.4, #x,y specifies the location of the center of the pie
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'), #the start and end angle of each slice
        line_color="white", fill_color='color', legend_field='Agency', source=data) #source is our data (can be a df or a CDS)

#Since there are way too many small slices, we'll only label slices with pct > 5
data["label_vals"] = np.where(data["pct"]>=5,data["pct"],0)                              
data["label_vals"]=data["label_vals"].astype(str).str.pad(35, side = "left") #pad will move the labels out toward the circumference
data["label_vals"]=data["label_vals"].apply(lambda x: "" if x.strip()=="0.0" else x)

#LabelSet is the set of labels. Either a value or nothing
labels = LabelSet(x=0, y=1, text='label_vals',
        angle=cumsum('angle', include_zero=True), source=ColumnDataSource(data), render_mode='canvas')

#Add the labels
p.add_layout(labels)


p.axis.axis_label=None #Piechart. No axes
p.axis.visible=False #We don't want to see the axes
p.grid.grid_line_color = None #If you want grid lines, specify a color

#Show the chart
show(p)



<br><br><br><br>
<span style="color:blue;font-size:large">Grid plots in bokeh</span>
<li>define each figure separately</li>
<li>use gridplot to create a grid</li>
<li>gridplot takes a matrix of plots as its input</li>

<br><br><br><br>
<span style="color:blue;font-size:large">Piecharts for all boroughs in a grid</span>
<li>pyplot has a "grid" feature</li>
<li>create grid specifications</li>
<li>add charts to each grid cell (they could be of different types)</li>
<li>We'll write a function that creates a piechart for each borough</li>

In [21]:
def create_bokeh_pi(df,row):
    from math import pi #Need this - a pie chart is a circle
    import pandas as pd

    from bokeh.palettes import Category20c #The color palette
    from bokeh.models import LabelSet, ColumnDataSource
    from bokeh.transform import cumsum #a bokeh function that cumulatively sums the angle column in a column data source
    borough = row

    #Create a dataframe with an Agency column and a value column for the borough
    data = size_df.loc[borough].reset_index(name='value')

    #We'll restrict the size of the string for Agencies. One Agency is way too long!
    data['Agency'] = data['Agency'].str.slice(0,8)

    #Calculate a percent column (the fraction of cases for each agency)
    data['pct'] = (data['value']/sum(data['value'])*100).round(2)

    #Calculate an angel column (the fraction of 360 degrees for each slice)
    data['angle'] = data['pct']/(data['pct'].sum()) * 2*pi

    #Specify the color palette
    data['color'] = Category20c[len(data)]

    #Define a figure object
    p = figure(plot_height=350, title="Cases by Agency: " + borough, 
            tools="hover", tooltips="@Agency: @pct",x_range=(-0.5, 1.0))

    #Add the definition of each wedge
    #Each wedge, starting with 0 degrees, will be at the sum of the angles upto that wedge

    p.wedge(x=0, y=1, radius=0.4, #x,y specifies the location of the center of the pie
            start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'), #the start and end angle of each slice
            line_color="white", fill_color='color', legend_field='Agency', source=data,) #source is our data (can be a df or a CDS)
            
    #Since there are way too many small slices, we'll only label slices with pct > 5
    data["label_vals"] = np.where(data["pct"]>5,data["pct"],0)                              
    data["label_vals"]=data["label_vals"].astype(str).str.pad(20, side = "left") #pad will move the labels out toward the circumference
    data["label_vals"]=data["label_vals"].apply(lambda x: "" if x.strip()=="0.0" else x)

    #LabelSet is the set of labels. Either a value or nothing
    labels = LabelSet(x=0, y=1, text='label_vals',
            angle=cumsum('angle', include_zero=True), source=ColumnDataSource(data), render_mode='canvas')

    #Add the labels
    p.add_layout(labels)


    p.axis.axis_label=None #Piechart. No axes
    p.axis.visible=False #We don't want to see the axes
    p.grid.grid_line_color = None #If you want grid lines, specify a color

    #Show the chart
    return p



In [22]:
plots=list()
for borough in size_df.index:
    plots.append(create_bokeh_pi(size_df,borough))
grid = gridplot(plots,sizing_mode="scale_both",merge_tools=True,ncols=2)
show(grid)

<br><br><br><br><br><br>
<span style="color:green;font-size:60px">Interactive charting</span>
<br><br>

<li>bokeh graphs are interactive</li>
<li>passive interaction (the graph does not change with user input) (e.g., the piechart above)</li>
<li>active interaction (the graph changes with user input)</li>

<br><br><br><br>
<span style="color:blue;font-size:x-large">distribution of processing time</span>
<li>We've looked at mean processing times over months and days</li>
<li>Let's get a sense of the distribution of processing times</li>

<br><br><br><br>
<span style="color:blue;font-size:large">Histogram of processing times</h2>
<li>We'll only keep data for which the processing duration is greater than .5 and less than 1 day </li>
<li>Why? No particular reason other than graph aesthetics!</li>
<li>And divide the data into ten bins</li>
<li>and use <a href="https://numpy.org/doc/stable/reference/generated/numpy.histogram.html">np.histogram</a> to create the histogram</li>

In [23]:
#Load the data
import pandas as pd
import numpy as np
df = pd.read_csv("../class-datasets/nyc_311_clean.csv",header=0, nrows=1000000)
df['Created Date'] = pd.to_datetime(df["Created Date"],format="%Y-%m-%d %H:%M:%S")
df['Closed Date'] = pd.to_datetime(df["Closed Date"],format="%Y-%m-%d %H:%M:%S")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype         
---  ------           --------------    -----         
 0   Created Date     1000000 non-null  datetime64[ns]
 1   Closed Date      1000000 non-null  datetime64[ns]
 2   Agency           1000000 non-null  object        
 3   Incident Zip     1000000 non-null  int64         
 4   Borough          1000000 non-null  object        
 5   Latitude         1000000 non-null  float64       
 6   Longitude        1000000 non-null  float64       
 7   processing_days  1000000 non-null  float64       
dtypes: datetime64[ns](2), float64(3), int64(1), object(2)
memory usage: 61.0+ MB


In [24]:
import numpy as np
import pandas as pd
df2 = df[(df['processing_days']>0.5) & (df['processing_days']<1)]
nbins=10
histogram,boundaries = np.histogram(df2['processing_days'],bins=nbins)
hist_df = pd.DataFrame({'cases': histogram, 
                       'left': boundaries[:-1]*24*60, 
                       'right': boundaries[1:]*24*60})

In [25]:
boundaries

array([0.5000463 , 0.55004051, 0.60003472, 0.65002894, 0.70002315,
       0.75001736, 0.80001157, 0.85000579, 0.9       , 0.94999421,
       0.99998843])

In [26]:
histogram

array([3218, 3166, 2919, 2848, 3306, 3451, 3853, 4299, 4605, 4477])

In [27]:
hist_df

,cases,left,right
0,3218,720.066667,792.058333
1,3166,792.058333,864.050000
2,2919,864.050000,936.041667
3,2848,936.041667,1008.033333
4,3306,1008.033333,1080.025000
5,3451,1080.025000,1152.016667
6,3853,1152.016667,1224.008333
7,4299,1224.008333,1296.000000
8,4605,1296.000000,1367.991667
9,4477,1367.991667,1439.983333


<br><br><br>
<span style="color:blue;font-size:x-large">Creating an interactive bokeh bar chart</span>
<li>We'll convert the dataframe into a ColumnDataSource object</li>
<li>Create a <span style="color:blue">tooltips</span> dictionary (for the data that will be shown when hovering)</li>
<li>Center the bar chart to the median bar size</li>
<li>Use a <a href="https://docs.bokeh.org/en/latest/docs/reference/models/glyphs/quad.html">quad glyph</a> to construct each bar. Quad = Quadrilateral</li>

In [28]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource

output_notebook()

Loading BokehJS ...

In [29]:
#Place the data into a ColumnDataSource object
src = ColumnDataSource(hist_df)

#create the tooltips 
#whatever you put in here will be shown when hovering
#the @ tells bokeh to use data from the columns of the columndatasource
#formatting goes in curly braces
tooltips = [
    ("number of cases", "@cases"),
    ("left (minutes)", "@left{s}"),
    ("right (minutes)", "@right{s}"),
]

"""
tooltips = {
    "number of cases": "@cases",
    "left (minutes)": "@left{s}",
    "right (minutes)": "@right{s}",
}
"""
#get the median for centering the x-axis
import statistics
median = statistics.median(src.data['cases'])

#Create the figure
p = figure(plot_height = 600, plot_width = 600, 
           title = 'Histogram of cases by processing times',
          x_axis_label = 'time buckets', 
           y_axis_label = 'number of cases',tooltips=tooltips)

# Add a quad glyph (https://docs.bokeh.org/en/latest/docs/reference/models/glyphs/quad.html)
#we'll use the source argument to attach the columndatasource to the chart
r = p.quad(bottom=median, #location of the x-axis
           top='cases',  #the data that will determine the height of the bar
           left='left', #CDS column for the left value of the bar
           right='right', #CDS column for the right value of the bar
           source=src,  #The ColumnDataSource (CDS)
           fill_color='red', 
           line_color='black',
           fill_alpha = 0.95,
           hover_fill_alpha = 1.0, 
           hover_fill_color = 'orange')



# Show the plot
show(p)

In [30]:
p,r

(Figure(id='3058', ...), GlyphRenderer(id='3098', ...))

<br><br><br>
<span style="color:blue;font-size:x-large">let's functionalize this</span>
<li>the function will return p (the plot)</li>
<li>and the "glyphrenderer" object</li>
<li>the glyphrenderer contains the data used by p</li>
<li>updating the data will result in the graph being updated which will be useful later</li>

<br><br><br>
<span style="color:green;font-size:xx-large">Active interactions</span>
<li>active interactions allow the user to delve into the data by selecting parameters</li>
<li>example, the user may want to bin by Agency (or Borough)</li>
<li>or select bin sizes rather than rely on our 500 minute window</li>
<li>or select periods of time and see how the histogram changes</li>

<li>In this example, we'll let the user:</li>
<ol>
    <li>select an agency from a list of agencies</li>
    <li>show binned processing time only for that agency</li>
</ol>
<li>the graph will change with each change in agency selection</li>

<br><br><br>
<span style="color:blue;font-size:x-large">JavaScript callbacks</span>
<li>changes to a page in a browser are made using JavaScript</li>
<li>we can include javascript widgets directly in the chart using bokeh's callback feature</li>
<li>the complication is that we can't call python functions from javascript</li>
<li>nor can we refer to a dataframe from JavaScript because there is no json equivalent</li>
<li>ColumnDataSource objects are fine though because they can be converted into JSON objects and arrays</li>
<li>And these json objects/arrays can be passed to a javascript function</li>

<b>Note:</b> While JavaScript is a full fledged language in its own right, we'll just use enough to make our graph interactive while keeping the process simple!

<br><br><br>
<span style="color:blue;font-size:x-large">columndatasource</span>
<li>each user selection (agency/bin size) requires data in a columndatasource object</li>
<li>so we need a function that creates this columndatasource given an agency and a bin size</li>


<br><br><br>
<span style="color:blue;font-size:x-large">dataframe for columndatasource</span>
<li>Create a dataframe that will be converted into a bokeh ColumnDataSource object</li>
<li>Each row corresponds to a bucket of the histogram</li>
<li>Columns correspond to agencies</li>
<li>Plus two additional columns for the left and right bin values for each bin</li>

In [31]:
#Create a dataframe for conversion to ColumnDataSource
#Upper and lower define the processing time bounds (aesthetic reasons!)

def prepareDataForCDS(df,upper=1,lower=0.5,nbins=10):
    df = df[(df['processing_days']<upper) & (df['processing_days']>lower)]
    agencies = df['Agency'].unique()
    
    #get a common set of bins for all agencies

    hist,bounds = np.histogram(df['processing_days'],bins=nbins)
    left = bounds[:-1]
    right = bounds[1:]
    
    #construct the initial dataframe with bin boundaries
    #the index for this df is 0,1,..,nbins-1
    #Initially, all it contains is columns for the left and right values
    #We'll add each agency column to this data frame
    final_df = pd.DataFrame({'left':left*24*60,'right':right*24*60})


    #for each agency, compute the number of cases in each bin
    #and concat this into final_df with the agency name as the column name
    #note that the index will be 0,1,...,nbins-1 therefore concat will use indices
    for agency in agencies:
        #print(final_df.info())
        a_df = df[df['Agency']==agency]
        hist,bounds = np.histogram(a_df['processing_days'],bins=nbins)
        hist_df = pd.DataFrame({agency: hist})
        final_df = pd.concat([final_df,hist_df],axis=1)
        
    return final_df



In [32]:
prepareDataForCDS(df,upper=1,lower=0.5,nbins=10)

,left,right,DHS,DOT,HPD,DSNY,NYPD,DEP,DOB,DPR,DCA,DOHMH,DOE,OTI,TLC
0,720.066667,792.058333,125,476,809,135,729,282,464,35,29,138,1,1,0
1,792.058333,864.050000,119,553,769,194,512,292,474,49,44,158,1,1,0
2,864.050000,936.041667,105,649,728,244,297,319,307,90,60,116,5,0,0
3,936.041667,1008.033333,84,650,839,328,157,366,121,108,67,129,5,0,0
4,1008.033333,1080.025000,126,848,935,469,147,334,46,158,97,129,2,0,0
5,1080.025000,1152.016667,103,968,985,592,79,334,8,126,133,115,7,0,1
6,1152.016667,1224.008333,80,1023,1175,709,97,431,7,114,117,87,6,1,0
7,1224.008333,1296.000000,74,1081,1385,808,178,409,7,130,133,86,10,1,0
8,1296.000000,1367.991667,53,1168,1505,960,65,437,28,156,130,96,5,0,0
9,1367.991667,1439.983333,56,960,1710,951,80,356,27,155,112,71,4,2,0


<br><br><br>
<span style="color:blue;font-size:x-large">Interactivity</span>
<li>Requires a dropdown box to get the user selection</li>
<li>Requires JavaScript code to change the underlying data when the user selects a new agency</li>
<li>The way this works is as follows:</li>
<ul>
    <li>Specify a column that contains the data to be displayed</li>
    <li>display this column</li>
    <li>when the user selects a new column, copy the contents of the new column into the display column</li>
    <li>and tell the graph to reflect the changed values</li>
</ul>
<li>We'll add a new column <span style="color:blue">display</span> for storing data to be displayed</li>
<li>We'll use the <span style="color:green">dropdown</span> widget to display the selectable list of agencies. Many more <a href="https://docs.bokeh.org/en/latest/docs/user_guide/interaction/widgets.html">widgets are available</a></li>

In [33]:
#row will allow placing the graph and the widget side by side
from bokeh.layouts import row

#import the dropdown widget along with CustomJS, bokeh's JavaScript library
from bokeh.models import CustomJS, ColumnDataSource, Dropdown

list_of_agencies=df['Agency'].unique()

#Get the dataframe
interactive_df = prepareDataForCDS(df)

#add a new column to the object called 'display'
#when the user selects a new agency, we'll copy that agency's data into this column
#the histogram will display whatever is in this column

#Arbitrarly pick an agency and copy it into this column

interactive_df['display'] = interactive_df['DOT']

#create a ColumnDataSource object.
#This is the source for the graph
source = ColumnDataSource(interactive_df)

#construct the graph (but don't "show" it)
#note that the display column is "display" not "cases" as before

tooltips = [
    ("number of cases", "@display"),
    ("left (minutes)", "@left{s}"),
    ("right (minutes)", "@right{s}"),
]
    
p = figure(plot_height = 300, plot_width = 600, 
           title = 'Histogram of cases by processing times',
          x_axis_label = 'time buckets', 
           y_axis_label = 'number of cases',tooltips=tooltips)

p.quad(bottom=0,
       top='display', left='left', right='right',source=source,
       fill_color='red', line_color='black',fill_alpha = 0.75,
       hover_fill_alpha = 1.0, hover_fill_color = 'navy')

#Create a javascript callback as a call to CustomJS
#arguments are the data (the source)
#and a string containing the JavaScript code

#this.item is the selected value (returned by the dropdown widget)
#"this" is a javascript identifier that self identifies an object (the dropdown widget)
#'item' is the value selected in the dropdown
#Note that different widgets may use this.value or this.some_other_name for a widget value
#create a variable that holds the data in source
#replace the display column in the data by the column for the selected agency
#register the change (emit it!)
jscallback = CustomJS(args={'srce':source},code="""
        // widget. this.item is the selected value
        // We can print it on the JavaScript console
        // Chrome windows/linux: Ctrl - Shift - J
        // Chrome mac: Cmd - Option - J
        // Safari: Option - Cmd - C
        
        //console.log logs (prints) in the JavaScript console
        console.log(' changed selected option', this.item);

        //data is the variable containing all the data
        //source was passed in as an argument
        let data = srce.data;

        // allocate the selected column to the field for the y values
        data['display'] = data[this.item];

        // register the change 
        srce.change.emit();
""")

#Create the dropdown widget
#the widget contains a menu in the form of a list of tuples
#(long name, value to be returned)
#the widget has a callback argument. Use the jscallback as the value of this argument

menu = [("Agency " + x,x) for x in list_of_agencies]

dropdown = Dropdown(label="select agency", button_type="primary", menu=menu)

#Note that jscallback must be defined before the button!
dropdown.js_on_event("menu_item_click", jscallback)

#construct a row object containing the plot and the widget
layout = row(p,dropdown)

#show it!
show(layout)



